# 第七章 贝叶斯分类器

## 7.1 试使用极大似然法估算西瓜数据集 3.0 中前3个属性的类条件概率。

In [1]:
import pandas as pd
import numpy as np

In [2]:
read_path = 'D:\JWE\Files\Courses\数据挖掘\melon_data.csv'
melon_data = pd.read_csv(read_path)
data0 = melon_data.copy() 
data0.loc[data0['好瓜'] == '是',['好瓜']] = '好瓜'
data0.loc[data0['好瓜'] == '否',['好瓜']] = '坏瓜'
character_dict = {character:attr for character,attr in zip(data0.columns.drop(['编号','好瓜']),[set(data0.loc[:,[each]].values.flatten()) for each in data0.columns.drop(['编号','好瓜'])])}
discrete_dict = {character:attr for character,attr in zip(data0.columns.drop(['密度','含糖率','编号','好瓜']),[set(data0.loc[:,[each]].values.flatten()) for each in data0.columns.drop(['密度','含糖率','编号','好瓜'])])}
continuous_dict = {character:character_dict[character] for character in character_dict.keys()-discrete_dict.keys()}

In [3]:
data0
discrete_dict
# continuous_dict

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,好瓜
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,好瓜
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,好瓜
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,好瓜
4,5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,好瓜
5,6,青绿,稍蜷,浊响,清晰,稍凹,软粘,0.403,0.237,好瓜
6,7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,0.481,0.149,好瓜
7,8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,好瓜
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,坏瓜
9,10,青绿,硬挺,清脆,清晰,平坦,软粘,0.243,0.267,坏瓜


{'色泽': {'乌黑', '浅白', '青绿'},
 '根蒂': {'硬挺', '稍蜷', '蜷缩'},
 '敲声': {'沉闷', '浊响', '清脆'},
 '纹理': {'模糊', '清晰', '稍糊'},
 '脐部': {'凹陷', '平坦', '稍凹'},
 '触感': {'硬滑', '软粘'}}

In [4]:
bys_dict = {f'{character}={attr}|{classi}':len(data0.loc[(data0[character]==attr)&(data0['好瓜']==classi),:]) for classi in set(data0.loc[:,['好瓜']].values.flatten()) for character in list(discrete_dict.keys()) for attr in discrete_dict[character]}
bys_dict
# len(bys_dict)

{'色泽=青绿|坏瓜': 3,
 '色泽=乌黑|坏瓜': 2,
 '色泽=浅白|坏瓜': 4,
 '根蒂=硬挺|坏瓜': 2,
 '根蒂=蜷缩|坏瓜': 3,
 '根蒂=稍蜷|坏瓜': 4,
 '敲声=沉闷|坏瓜': 3,
 '敲声=浊响|坏瓜': 4,
 '敲声=清脆|坏瓜': 2,
 '纹理=清晰|坏瓜': 2,
 '纹理=模糊|坏瓜': 3,
 '纹理=稍糊|坏瓜': 4,
 '脐部=凹陷|坏瓜': 2,
 '脐部=平坦|坏瓜': 4,
 '脐部=稍凹|坏瓜': 3,
 '触感=软粘|坏瓜': 3,
 '触感=硬滑|坏瓜': 6,
 '色泽=青绿|好瓜': 3,
 '色泽=乌黑|好瓜': 4,
 '色泽=浅白|好瓜': 1,
 '根蒂=硬挺|好瓜': 0,
 '根蒂=蜷缩|好瓜': 5,
 '根蒂=稍蜷|好瓜': 3,
 '敲声=沉闷|好瓜': 2,
 '敲声=浊响|好瓜': 6,
 '敲声=清脆|好瓜': 0,
 '纹理=清晰|好瓜': 7,
 '纹理=模糊|好瓜': 0,
 '纹理=稍糊|好瓜': 1,
 '脐部=凹陷|好瓜': 5,
 '脐部=平坦|好瓜': 0,
 '脐部=稍凹|好瓜': 3,
 '触感=软粘|好瓜': 2,
 '触感=硬滑|好瓜': 6}

这里只计算第一个属性$(色泽=attr|好瓜)$的类条件概率，假设该属性条件分布如下
$$
P(色泽=浅白|好瓜=是) = p_{1}\\
P(色泽=乌黑|好瓜=是) = p_{2}\\
P(色泽=青绿|好瓜=是) = p_{3} = 1 - p_{1} - p_{2}\\
$$
该属性各取值的条件概率极大似然函数为
$$
L 
= P(色泽=浅白|好瓜=是)^{1} \cdot P(色泽=乌黑|好瓜=是)^{4} \cdot P(色泽=青绿|好瓜=是)^{3} 
= p_1^1 \cdot p_2^4 \cdot (1 - p_1 - p_2)^3
$$
取对数，求偏导，找极值
$$
令
\begin{cases}
\frac{\partial ln(L)}{\partial p_1} = 0 \\
\frac{\partial ln(L)}{\partial p_2} = 0 \\
\end{cases}
$$
求得
$$
p_1 = \frac{1}{8} \\
p_2 = \frac{1}{2} \\
p_3 = \frac{3}{8} \\
$$

## 7.2.试证明：条件独立性假设不成立时，朴素贝叶斯分类器任有可能产生最优分类器。

假设条件独立性不成立，朴素贝叶斯分类器仍按条件独立进行。

此时若数据集无冲突数据，且具有相同属性取值的样本类别相同，不同属性取值的样本类别不同，则朴素贝叶斯分类器仍产生最优分类器。

## 7.3.试编程实现拉普拉斯修正的朴素贝叶斯分类器，并以西瓜数据集3.0为训练集，并对“测1”样本进行分类。

沿用上面代码

In [5]:
# data0
# discrete_dict
# continuous_dict
test_dict = {'色泽':['青绿'],'根蒂':['蜷缩'],'敲声':['浊响'],'纹理':['清晰'],'脐部':['凹陷'],'触感':['硬滑'],'密度':[0.697],'含糖率':[0.460],'好瓜':['？']}
test1 = pd.DataFrame(data=test_dict)
test1

,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
0,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.46,？


In [6]:
def Normpdf(x,mu=0,sigma=1):
    return np.exp(-((x - mu)**2)/(2*sigma**2))/(np.sqrt(2*np.pi)*sigma)

def SimpleBayes(data=data0,simple=test1):
    simple_dict = {f'{character}':f'{character}={attr}' for character,attr in zip(simple.columns.drop(['好瓜']),simple.drop(['好瓜'],axis=1).values.flatten())}
    simpleD_dict = {f'{character}':f'{character}={attr}' for character,attr in zip(simple.columns.drop(['密度','含糖率','好瓜']),simple.drop(['密度','含糖率','好瓜'],axis=1).values.flatten())}
    simpleC_dict = {character:simple_dict[character] for character in simple_dict.keys()-simpleD_dict.keys()}
    character_dict = {character:attr for character,attr in zip(data.columns.drop(['编号','好瓜']),[set(data.loc[:,[each]].values.flatten()) for each in data.columns.drop(['编号','好瓜'])])}
    discrete_dict = {character:attr for character,attr in zip(data.columns.drop(['密度','含糖率','编号','好瓜']),[set(data.loc[:,[each]].values.flatten()) for each in data.columns.drop(['密度','含糖率','编号','好瓜'])])}
    continuous_dict = {character:character_dict[character] for character in character_dict.keys()-discrete_dict.keys()}
    bayesD_dict = {}
    bayesC_dict = {}
    for classi in set(data.loc[:,['好瓜']].values.flatten()):
        bayesD_dict[classi] = {f'{character}={attr}':(len(data.loc[(data[character]==attr)&(data['好瓜']==classi),:])+1)/(len(data.loc[data['好瓜']==classi,:])+len(discrete_dict[character])) for character in list(discrete_dict.keys()) for attr in discrete_dict[character]}
        bayesC_dict[classi] = {f'{character}={attr}':Normpdf(attr,float(data.loc[data['好瓜']==classi,[character]].mean().values),float(data.loc[data['好瓜']==classi,[character]].std().values)) for character,attr in zip(continuous_dict.keys(),simple.loc[:,['密度','含糖率']].values.flatten())}
#     bayes_dict = dict(bayesD_dict,**bayesC_dict)
    PT = (len(data.loc[data['好瓜']=='好瓜',:])+1)/(len(data)+len(set(data['好瓜'].values.flatten())))
    PF = (len(data.loc[data['好瓜']=='坏瓜',:])+1)/(len(data)+len(set(data['好瓜'].values.flatten())))
    for index in simpleD_dict.keys():
        PT *= bayesD_dict['好瓜'][simpleD_dict[index]]
    for index in simpleC_dict.keys():
        PT *= bayesC_dict['好瓜'][simpleC_dict[index]]
    for index in simpleD_dict.keys():
        PF *= bayesD_dict['坏瓜'][simpleD_dict[index]]
    for index in simpleC_dict.keys():
        PF *= bayesC_dict['坏瓜'][simpleC_dict[index]]
    MLE_dict = {'好瓜':PT,'坏瓜':PF}
#     print(MLE_dict)
    result = max(MLE_dict,key=MLE_dict.get)
    return result,MLE_dict
#     return bayesD_dict,bayesC_dict,simple_dict

In [7]:
SimpleBayes(data0)

('好瓜', {'好瓜': 0.025631024529740677, '坏瓜': 7.722360621178051e-05})

## 7.4.实践中用式(7.15)决定分类类别时，若数据的维度非常高，则连乘的概率结果会非常接近0并导致下溢。试述防止下溢的可能方案。

把概率取对数，则连乘变为连加。

## 7.5.试证明:二分类任务中两类数据满足高斯分布且方差相同时，线性判别分析产生最优贝叶斯分类器。

日后再补

## 7.6.试编程实现AODE分类器，并以西瓜数据集3.0为训练集，并对“测1”样本进行分类。

日后再补

## 7.7.给定d个二值属性的分类任务，假设对于任何先验概率的估算需要30个样本。试估计AODE中估算先验概率p(c,xi)所需要的样本数。

日后再补

## 7.8.考虑图7.3，证明：在同父结构中，若x1的取值未知，则x3⊥x4不成立。在顺序结构中，y⊥z|x成立，但y⊥z不成立。

日后再补

## 7.9 以西瓜数据集 2.0 为训练集，试基于 BIC 准则构建一个贝叶斯网。

日后再补

## 7.10 以西瓜数据集 2.0 中属性"脐部"为隐变量，试基于 EM 算法构建一个贝叶斯网。

日后再补